In [1]:
import random
from itertools import combinations

## Имитационная модель сети Петри

In [2]:
class PetriNet:
    def __init__(self, transitions, marking):
        self.places = list([Place(m, i) for i, m in enumerate(marking)])
        self.transitions = list([
            Transition(
                [OutArc(self.places[i]) for i in outs], 
                [InArc(self.places[i]) for i in ins]) 
            for outs, ins in transitions
        ])
        self.finished = False
        self.markings = [self.get_marking()]

    def get_marking(self):
        return [x.holding for x in self.places]

    def run(self, verbose=False):
        i = 0
        while not self.finished and i < 25:
            current_marking = self._run_once(verbose)
            if not current_marking:
                self.finished = True
                break
            yield current_marking
            i += 1

    def _run_once(self, verbose):
        fired_transitions = set()
        while len(fired_transitions) < len(self.transitions):
            t_num = random.randint(0, len(self.transitions) - 1)
            t = self.transitions[t_num]
            if t.fire():
                if verbose:
                    print(f"Firing transition #{t_num}: {t}")
                    print(f"Current marking {self.get_marking()}")
                return self.get_marking()
            else:
                fired_transitions.add(t_num)
        return None


class Transition:
    def __init__(self, out_arcs, in_arcs):
        self.out_arcs = set(out_arcs)
        self.in_arcs = set(in_arcs)
        self.arcs = self.out_arcs.union(in_arcs)

    def fire(self):
        if self.not_blocked():
            for arc in self.arcs:
                arc.trigger()
            return True
        return False

    def not_blocked(self):
        return all(arc.non_blocking() for arc in self.out_arcs)

    def get_out_arc(self, place):
        return [a for a in self.out_arcs if a.place == place][0]

    def get_in_arc(self, place):
        return [a for a in self.in_arcs if a.place == place][0]

    def __str__(self):
        return f"{list(self.out_arcs)} -> {list(self.in_arcs)}"

    __repr__ = __str__


class Place:
    def __init__(self, holding, id=None):
        self.holding = holding
        self.id = id

    def __str__(self):
        return f"#{self.id}({self.holding})"

    __repr__ = __str__


class Arc:
    def __init__(self, place: Place, amount=1):
        self.place = place
        self.amount = amount

    def __str__(self):
        return f"{self.place}" + (f"[{self.amount}]" if self.amount > 1 else "")

    __repr__ = __str__


class OutArc(Arc):
    def trigger(self):
        self.place.holding -= self.amount

    def non_blocking(self):
        return self.place.holding >= self.amount


class InArc(Arc):
    def trigger(self):
        self.place.holding += self.amount

## Исследуемые сети

1) ![alt text](https://imgur.com/uVBQESS.png)
   
   
2) ![alt text](https://imgur.com/2311NKm.png)

In [3]:
# 1
marking1 = [0, 0, 1, 1, 0]
transitions1 = [
    ([3], [0]),
    ([2], [1]),
    ([0, 1], [2, 3]),
    ([3], [4])
]

In [4]:
#2
marking2 = [1, 0, 0, 0, 0, 1]
transitions2 = [
    ([0], [1, 2, 5]),
    ([1], [3]),
    ([2], [4]),
    ([3, 4, 5], [0])
]

## Характеристики сети Петри

### Ограниченность

In [5]:
def bounded_with(net: PetriNet):
    markings = [net.get_marking()]
    markings += [m for m in net.run()]
    return max([max(m) for m in markings])

In [6]:
print(bounded_with(PetriNet(transitions1, marking1)))

1


### Безопасность

In [7]:
def is_safe(net: PetriNet):
    return bounded_with(net) == 1

In [8]:
print(is_safe(PetriNet(transitions1, marking1)))

True


### Консервативность

In [9]:
def is_conservative(net: PetriNet):
    s0 = sum(net.get_marking())
    for marking in net.run():
        s = sum(marking)
        if s != s0: return False
    return True

In [10]:
print(is_conservative(PetriNet(transitions1, marking1)))

True


### Живая / не живая

In [11]:
def is_alive(net: PetriNet):
    for _ in net.run(): pass
    return not net.finished

In [12]:
print(is_alive(PetriNet(transitions1, marking1)))

False


### Параллельность

In [13]:
def check_parralel(net: PetriNet):
    not_blocked_ts = [t for t in net.transitions if t.not_blocked()]
    if len(not_blocked_ts) <= 1:
        return False
    for t1, t2 in combinations(not_blocked_ts, 2):
        places1 = set(place for place in t1.out_arcs)
        places2 = set(place for place in t2.out_arcs)
        if places1 != places2:
            return True
        mutual_places = places1.intersection(places2)
        if all(p.holding >= t1.get_out_arc(p).amount + t2.get_out_arc(p).amount for p in mutual_places):
            return True
    return False

def is_parallel(net: PetriNet):
    if check_parralel(net):
        return True
    for _ in net.run():
        if check_parralel(net):
            return True
    return False

In [14]:
print(is_parallel(PetriNet(transitions1, marking1)))

True


### Свободного выбора

In [15]:
def is_free_choice(net: PetriNet):
    for t1, t2 in combinations(net.transitions, 2):
        places1 = set(arc.place for arc in t1.out_arcs)
        places2 = set(arc.place for arc in t2.out_arcs)
        if places1.intersection(places2):
            return True
    return False

In [16]:
print(is_free_choice(PetriNet(transitions1, marking1)))

True


### Маркированная

In [17]:
def is_marked(net: PetriNet):
    for place in net.places:
        in_arcs = [a for t in net.transitions 
            for a in t.in_arcs if a.place == place]
        out_arcs = [a for t in net.transitions 
            for a in t.out_arcs if a.place == place]
        if len(out_arcs) != len(in_arcs) != 1:
            return False
    return True

In [18]:
print(is_free_choice(PetriNet(transitions1, marking1)))

True


### Полное исследование сети #1

![](https://imgur.com/uVBQESS.png)

In [19]:
def net1():
    return PetriNet(transitions1, marking1)

for _ in net1().run(verbose=True): pass

print()
print(f"К-bounded: {bounded_with(net1())}")
print(f"Is safe: {is_safe(net1())}")
print(f"Is conservative: {is_conservative(net1())}")
print(f"Is alive: {is_alive(net1())}")
print(f"Is parallel: {is_parallel(net1())}")
print(f"Is free choice: {is_free_choice(net1())}")
print(f"Is marked: {is_marked(net1())}")

Firing transition #1: [#2(0)] -> [#1(1)]
Current marking [0, 1, 0, 1, 0]
Firing transition #0: [#3(0)] -> [#0(1)]
Current marking [1, 1, 0, 0, 0]
Firing transition #2: [#1(0), #0(0)] -> [#2(1), #3(1)]
Current marking [0, 0, 1, 1, 0]
Firing transition #1: [#2(0)] -> [#1(1)]
Current marking [0, 1, 0, 1, 0]
Firing transition #0: [#3(0)] -> [#0(1)]
Current marking [1, 1, 0, 0, 0]
Firing transition #2: [#1(0), #0(0)] -> [#2(1), #3(1)]
Current marking [0, 0, 1, 1, 0]
Firing transition #0: [#3(0)] -> [#0(1)]
Current marking [1, 0, 1, 0, 0]
Firing transition #1: [#2(0)] -> [#1(1)]
Current marking [1, 1, 0, 0, 0]
Firing transition #2: [#1(0), #0(0)] -> [#2(1), #3(1)]
Current marking [0, 0, 1, 1, 0]
Firing transition #1: [#2(0)] -> [#1(1)]
Current marking [0, 1, 0, 1, 0]
Firing transition #3: [#3(0)] -> [#4(1)]
Current marking [0, 1, 0, 0, 1]

К-bounded: 1
Is safe: True
Is conservative: True
Is alive: False
Is parallel: True
Is free choice: True
Is marked: True


### Полное исследование сети #2

![](https://imgur.com/2311NKm.png)

In [20]:
def net2():
    return PetriNet(transitions2, marking2)

for _ in net2().run(verbose=True): pass

print()
print(f"К-bounded: {bounded_with(net2())}")
print(f"Is safe: {is_safe(net2())}")
print(f"Is conservative: {is_conservative(net2())}")
print(f"Is alive: {is_alive(net2())}")
print(f"Is parallel: {is_parallel(net2())}")
print(f"Is free choice: {is_free_choice(net2())}")
print(f"Is marked: {is_marked(net2())}")

Firing transition #0: [#0(0)] -> [#1(1), #2(1), #5(2)]
Current marking [0, 1, 1, 0, 0, 2]
Firing transition #1: [#1(0)] -> [#3(1)]
Current marking [0, 0, 1, 1, 0, 2]
Firing transition #2: [#2(0)] -> [#4(1)]
Current marking [0, 0, 0, 1, 1, 2]
Firing transition #3: [#5(1), #4(0), #3(0)] -> [#0(1)]
Current marking [1, 0, 0, 0, 0, 1]
Firing transition #0: [#0(0)] -> [#1(1), #2(1), #5(2)]
Current marking [0, 1, 1, 0, 0, 2]
Firing transition #2: [#2(0)] -> [#4(1)]
Current marking [0, 1, 0, 0, 1, 2]
Firing transition #1: [#1(0)] -> [#3(1)]
Current marking [0, 0, 0, 1, 1, 2]
Firing transition #3: [#5(1), #4(0), #3(0)] -> [#0(1)]
Current marking [1, 0, 0, 0, 0, 1]
Firing transition #0: [#0(0)] -> [#1(1), #2(1), #5(2)]
Current marking [0, 1, 1, 0, 0, 2]
Firing transition #2: [#2(0)] -> [#4(1)]
Current marking [0, 1, 0, 0, 1, 2]
Firing transition #1: [#1(0)] -> [#3(1)]
Current marking [0, 0, 0, 1, 1, 2]
Firing transition #3: [#5(1), #4(0), #3(0)] -> [#0(1)]
Current marking [1, 0, 0, 0, 0, 1]
Firi

In [0]:
marking = []
transitions = []
try:
    net_number = int(input('Choose a net (1-2): '))
    if net_number == 1:
        marking = [0, 0, 1, 1, 0]
        transitions = [
            ([3], [0]),
            ([2], [1]),
            ([0, 1], [2, 3]),
            ([3], [4])
        ]
    elif net_number == 2:
        marking = [1, 0, 0, 0, 0, 1]
        transitions = [
            ([0], [1, 2, 5]),
            ([1], [3]),
            ([2], [4]),
            ([3, 4, 5], [0])
        ]
    else:
        raise Exception('Net not found')
except Exception:
    print('Input error')
    exit(0)

print(marking)
print(transitions)
while True:
    print('\nUpdate marking - 1,\n'
          'Add new position - 2,\n'
          'Add new transition - 3,\n'
          'Add In arc to a transition - 4,\n'
          'Add Out arc to a transition - 5,\n'
          'Remove transition - 6, \n'
          'Run current net - 7,\n'
          'Exit - 0: ')
    add_answer = input()
    try:
        # Updating marking
        if add_answer == '1':
            print(f'Current marking: {marking}')
            print(f'Enter: \n1. Position number (0-{len(marking) - 1}), \n2. Number of marks: ')
            m_index, m_count = input().split(',')
            marking[int(m_index)] = int(m_count)
            print(f'New marking: {marking}')
            print(f'Current net: {transitions}\n')

        # Adding position
        elif add_answer == '2':
            marking.append(0)
            print(f'Current marking: {marking}')
            print(f'Current net: {transitions}\n')

        # Adding transition
        elif add_answer == '3':
            print(f"Enter list of In positions for new transition (0-{len(marking) - 1})")
            v_in = [int(x) for x in input().split(",")]

            print(f"Enter list of Out positions for new transition (0-{len(marking) - 1})")
            v_out = [int(x) for x in input().split(",")]

            transitions.append((v_in, v_out))
            print(f'Current marking: {marking}')
            print(f'Current net: {transitions}\n')

        # Adding In arc to transition
        elif add_answer == '4':
            print(f"Enter index of transition (0-{len(transitions) - 1})")
            t_index = int(input())

            print(f"Enter index of position (0-{len(marking) - 1})")
            p_index = int(input())

            transitions[t_index][0].append(p_index)
            print(f'Curernt marking: {marking}')
            print(f'Current net: {transitions}\n')
            
        # Adding Out arc to transition
        elif add_answer == '5':
            print(f"Enter index of transition (0-{len(transitions) - 1})")
            t_index = int(input())

            print(f"Enter index of position (0-{len(marking) - 1})")
            p_index = int(input())

            transitions[t_index][1].append(p_index)
            print(f'Curernt marking: {marking}')
            print(f'Current net: {transitions}\n')

        # Removing transition
        elif add_answer == '6':
            print(f"Enter index of transition (0-{len(transitions) - 1})")
            t_index = int(input())
            transitions.pop(t_index)
            
        # Running net
        elif add_answer == '7':
            def net():
                return PetriNet(transitions, marking)

            for _ in net().run(verbose=True): pass
                
            print()
            print(f"К-bounded: {bounded_with(net())}")
            print(f"Is safe: {is_safe(net())}")
            print(f"Is conservative: {is_conservative(net())}")
            print(f"Is alive: {is_alive(net())}")
            print(f"Is parallel: {is_parallel(net())}")
            print(f"Is free choice: {is_free_choice(net())}")
            print(f"Is marked: {is_marked(net())}")

        # Exit
        elif add_answer == '0':
            exit(0)

    except Exception:
        print('Input error')
        continue